#**Analisis Sentimen Pengguna terhadap Aplikasi Grab**

In [35]:
!pip install Sastrawi
!pip install nltk
!pip install numpy

import numpy as np
import pandas as pd
import tensorflow as tf
import re
import nltk
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.tokenize import word_tokenize

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [36]:
from google.colab import files
uploaded = files.upload()

Saving grabreviews.csv to grabreviews.csv


Data Cleaning

In [37]:
import pandas as pd
df = pd.read_csv('/content/grabreviews.csv')

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   content   15000 non-null  object
 1   score     15000 non-null  int64 
 2   sentimen  15000 non-null  object
dtypes: int64(1), object(2)
memory usage: 351.7+ KB


In [39]:
df.head()

,content,score,sentimen
0,"gak jelassss,, cari lokasi jemput harus susah ...",1,negative
1,enak sangat memuaskan kan reccoment banget,5,positive
2,sangat membantu,5,positive
3,game bus telolet,3,neutral
4,aku sih ok banget pelayanan nya,5,positive


In [40]:
df.isnull().sum()

,0
content,0
score,0
sentimen,0


In [41]:
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def clean_text(teks):
    """
    Membersihkan teks dari karakter yang tidak relevan seperti mention, hashtag, angka, dan simbol.
    """
    teks = re.sub(r'@\w+', '', teks)               # hapus mention
    teks = re.sub(r'#\w+', '', teks)               # hapus hashtag
    teks = re.sub(r'\bRT\b', '', teks)             # hapus RT
    teks = re.sub(r'http\S+', '', teks)            # hapus link
    teks = re.sub(r'\d+', '', teks)                # hapus angka
    teks = re.sub(r'[^\w\s]', '', teks)            # hapus tanda baca & simbol
    teks = re.sub(r'[^\x00-\x7F]+', ' ', teks)     # hapus emoji / karakter asing
    teks = teks.replace('\n', ' ').strip()         # hilangkan newline & spasi berlebih
    return teks


def lowercase_text(teks):
    return teks.lower().strip()

def tokenize_text(teks):
    """
    Memecah kalimat menjadi token kata.
    """
    return word_tokenize(teks)

def remove_stopwords(kata_list):
    """
    Menghapus stopwords bahasa Indonesia & Inggris + kata tidak penting lainnya.
    """
    stop_id = set(stopwords.words('indonesian'))
    stop_en = set(stopwords.words('english'))
    tambahan_stop = {
        'iya','yaa','gak','ga','nga','nya','na','sih','ku','nih','tuh','deh',
        'dong','loh','lah','ya','yah','okee','okey','oke','ok',
        'seller','barangnya','produknya','tokonya','jual','jualnya',
        'pengiriman','pengirimannya','kurir','pesanan','pesen','pesennya',
        'packing','dikemas','kemas','kemasnya','udah','sudah','lagi','bgt',
        'banget','bngt','btw','kk','kak','gan','sis','bro','woi','woii','woy',
        'barang','produk','toko','seller','pembeli','penjual','cepet','cepat',
        'lama','murah','mahal','mantap','bagus','baik','recommended','recomended'
    }

    all_stopwords = stop_id.union(stop_en).union(tambahan_stop)
    hasil = [kata for kata in kata_list if kata not in all_stopwords]
    return hasil

def stem_text(teks):
    """
    Mengembalikan kata ke bentuk dasar menggunakan Sastrawi Stemmer.
    """
    stemmer = StemmerFactory().create_stemmer()
    kata = teks.split()
    hasil = [stemmer.stem(k) for k in kata]
    return ' '.join(hasil)

def list_to_sentence(token_list):
    return ' '.join(token_list)


In [42]:
def convert_slang(teks_asli):

    kamus_slang = {
        "bgt": "banget", "bgtt": "banget", "bgttt": "banget",
        "bangett": "banget", "bangettt": "banget", "bngt": "banget",
        "pokonya": "pokoknya", "pokokny": "pokoknya", "pokoknyaa": "pokoknya",
        "mantab": "mantap", "mantabb": "mantap", "mantul": "mantap betul",
        "baguss": "bagus", "bagusss": "bagus", "seneng": "senang",
        "nyaman banget": "sangat nyaman", "cepet": "cepat",
        "cepet banget": "sangat cepat", "gk": "tidak", "gak": "tidak",
        "ga": "tidak", "ngga": "tidak", "nggak": "tidak",
        "jd": "jadi", "jg": "juga", "km": "kamu", "kmu": "kamu",
        "ak": "saya", "aq": "saya", "sy": "saya", "aku": "saya",
        "udh": "sudah", "udah": "sudah", "udhh": "sudah",
        "makasih": "terima kasih", "mksh": "terima kasih", "tq": "terima kasih",
        "thx": "terima kasih", "thanks": "terima kasih", "trims": "terima kasih",
        "blm": "belum", "dr": "dari", "dgn": "dengan", "dtg": "datang",
        "sampe": "sampai", "sampek": "sampai", "rekomen": "rekomendasi",
        "rekomended": "direkomendasikan", "recommended": "direkomendasikan",
        "barang2": "barang", "muas": "puas", "brg": "barang",
        "mantull": "mantap", "mantull banget": "mantap banget",
        "good": "bagus", "best": "terbaik", "simple": "sederhana",
        "ok": "baik", "okee": "baik", "okey": "baik"
    }

    teks = re.sub(r'\s+', ' ', teks_asli.lower().strip())

    # pisahkan per kata agar bisa dicek satu-satu
    kata2 = teks.split()

    hasil = []
    for kata in kata2:
        # jika slang ada di kamus, ubah ke versi baku
        if kata in kamus_slang:
            hasil.append(kamus_slang[kata])
        else:
            hasil.append(kata)

    # gabungkan kembali
    teks_baru = " ".join(hasil)
    return teks_baru

Pipeline Preprocessing Dataset Grab Reviews

In [43]:
from nltk.tokenize import word_tokenize

# --- Bersihkan teks dari simbol dan karakter aneh ---
df['clean_text'] = df['content'].astype(str).apply(clean_text)

# --- Jadikan semua huruf kecil (casefolding) ---
df['lower_text'] = df['clean_text'].apply(lowercase_text)

# --- Normalisasi kata gaul / tidak baku ---
df['normalized_text'] = df['lower_text'].apply(convert_slang)

# --- Tokenisasi (pisah jadi per kata) ---
df['tokenized_text'] = df['normalized_text'].apply(tokenize_text)

# --- Hapus stopwords (kata yang tidak penting) ---
df['filtered_text'] = df['tokenized_text'].apply(remove_stopwords)

# --- Gabungkan kembali token jadi satu kalimat utuh ---
df['final_text'] = df['filtered_text'].apply(list_to_sentence)

# --- Tampilkan hasil akhir untuk verifikasi ---
print(df[['content', 'final_text']].head(15))


                                              content  \
0   gak jelassss,, cari lokasi jemput harus susah ...   
1          enak sangat memuaskan kan reccoment banget   
2                                     sangat membantu   
3                                    game bus telolet   
4                     aku sih ok banget pelayanan nya   
5   KL pas update lama sekali padahal udh full 100...   
6                              tidak boleh di upgrade   
7                                         apk nya oke   
8   Kenapa limit paylater saya hilang,padahal pemb...   
9                                                Baik   
10  keluh kesah driver tolong didengarkan dan di p...   
11                                               😅lll   
12  lama banget, hampir 2 jam pesanan ga sampai sa...   
13                                    sangat membantu   
14                         dapat driver lama sekali .   

                                           final_text  
0             jelassss cari lok

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   content          15000 non-null  object
 1   score            15000 non-null  int64 
 2   sentimen         15000 non-null  object
 3   clean_text       15000 non-null  object
 4   lower_text       15000 non-null  object
 5   normalized_text  15000 non-null  object
 6   tokenized_text   15000 non-null  object
 7   filtered_text    15000 non-null  object
 8   final_text       15000 non-null  object
dtypes: int64(1), object(8)
memory usage: 1.0+ MB


In [45]:
df

,content,score,sentimen,clean_text,lower_text,normalized_text,tokenized_text,filtered_text,final_text
0,"gak jelassss,, cari lokasi jemput harus susah ...",1,negative,gak jelassss cari lokasi jemput harus susah ny...,gak jelassss cari lokasi jemput harus susah ny...,tidak jelassss cari lokasi jemput harus susah ...,"[tidak, jelassss, cari, lokasi, jemput, harus,...","[jelassss, cari, lokasi, jemput, susah, ampun]",jelassss cari lokasi jemput susah ampun
1,enak sangat memuaskan kan reccoment banget,5,positive,enak sangat memuaskan kan reccoment banget,enak sangat memuaskan kan reccoment banget,enak sangat memuaskan kan reccoment banget,"[enak, sangat, memuaskan, kan, reccoment, banget]","[enak, memuaskan, reccoment]",enak memuaskan reccoment
2,sangat membantu,5,positive,sangat membantu,sangat membantu,sangat membantu,"[sangat, membantu]",[membantu],membantu
3,game bus telolet,3,neutral,game bus telolet,game bus telolet,game bus telolet,"[game, bus, telolet]","[game, bus, telolet]",game bus telolet
4,aku sih ok banget pelayanan nya,5,positive,aku sih ok banget pelayanan nya,aku sih ok banget pelayanan nya,saya sih baik banget pelayanan nya,"[saya, sih, baik, banget, pelayanan, nya]",[pelayanan],pelayanan
...,...,...,...,...,...,...,...,...,...
14995,"sangat membantu, terimakasih",5,positive,sangat membantu terimakasih,sangat membantu terimakasih,sangat membantu terimakasih,"[sangat, membantu, terimakasih]","[membantu, terimakasih]",membantu terimakasih
14996,oke keren,5,positive,oke keren,oke keren,oke keren,"[oke, keren]",[keren],keren
14997,sangat membantu,5,positive,sangat membantu,sangat membantu,sangat membantu,"[sangat, membantu]",[membantu],membantu
14998,baik ..x ramah lagi,5,positive,baik x ramah lagi,baik x ramah lagi,baik x ramah lagi,"[baik, x, ramah, lagi]","[x, ramah]",x ramah


Labeling Dataset

In [49]:
import csv
import requests
from io import StringIO

def load_lexicon_from_url(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = {}

        csv_text = StringIO(response.text)
        reader = csv.reader(csv_text)

        for row in reader:
            if len(row) >= 2:
                kata = row[0].strip()
                skor = row[1].strip()
                try:
                    data[kata] = int(skor)
                except ValueError:
                    continue  # lewati jika skor bukan angka

        return data

    except requests.RequestException as e:
        print(f"⚠️ Gagal memuat data dari {url}: {e}")
        return {}

# URL dataset
url_pos = "https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_positive.csv"
url_neg = "https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_negative.csv"

# Muat kedua lexicon
lexicon_pos = load_lexicon_from_url(url_pos)
lexicon_neg = load_lexicon_from_url(url_neg)

print(f"✅ Lexicon positif berhasil dimuat: {len(lexicon_pos)} kata")
print(f"✅ Lexicon negatif berhasil dimuat: {len(lexicon_neg)} kata")


✅ Lexicon positif berhasil dimuat: 3609 kata
✅ Lexicon negatif berhasil dimuat: 6607 kata


In [50]:
def analyze_sentiment_lexicon(tokens):

    if not tokens or not isinstance(tokens, list):
        return 0, "neutral"

    total_score = 0

    for token in tokens:
        kata = token.lower()
        if kata in lexicon_pos:
            total_score += lexicon_pos[kata]
        elif kata in lexicon_neg:
            total_score += lexicon_neg[kata]

    if total_score > 0:
        sentiment = "positive"
    elif total_score < 0:
        sentiment = "negative"
    else:
        sentiment = "neutral"

    return total_score, sentiment


In [51]:
sentiment_result = df['filtered_text'].apply(analyze_sentiment_lexicon)

# Ubah hasil tuple (score, label) menjadi dua kolom terpisah
df[['sentiment_score', 'sentiment_label']] = pd.DataFrame(
    sentiment_result.tolist(), index=df.index
)

print("Distribusi sentimen:")
print(df['sentiment_label'].value_counts())

df.head()

Distribusi sentimen:
sentiment_label
neutral     6735
positive    6610
negative    1655
Name: count, dtype: int64


,content,score,sentimen,clean_text,lower_text,normalized_text,tokenized_text,filtered_text,final_text,sentiment_score,sentiment_label
0,"gak jelassss,, cari lokasi jemput harus susah ...",1,negative,gak jelassss cari lokasi jemput harus susah ny...,gak jelassss cari lokasi jemput harus susah ny...,tidak jelassss cari lokasi jemput harus susah ...,"[tidak, jelassss, cari, lokasi, jemput, harus,...","[jelassss, cari, lokasi, jemput, susah, ampun]",jelassss cari lokasi jemput susah ampun,-5,negative
1,enak sangat memuaskan kan reccoment banget,5,positive,enak sangat memuaskan kan reccoment banget,enak sangat memuaskan kan reccoment banget,enak sangat memuaskan kan reccoment banget,"[enak, sangat, memuaskan, kan, reccoment, banget]","[enak, memuaskan, reccoment]",enak memuaskan reccoment,10,positive
2,sangat membantu,5,positive,sangat membantu,sangat membantu,sangat membantu,"[sangat, membantu]",[membantu],membantu,4,positive
3,game bus telolet,3,neutral,game bus telolet,game bus telolet,game bus telolet,"[game, bus, telolet]","[game, bus, telolet]",game bus telolet,2,positive
4,aku sih ok banget pelayanan nya,5,positive,aku sih ok banget pelayanan nya,aku sih ok banget pelayanan nya,saya sih baik banget pelayanan nya,"[saya, sih, baik, banget, pelayanan, nya]",[pelayanan],pelayanan,3,positive


In [52]:
df.to_excel("GrabReviews_Clean.xlsx", index=False)

In [4]:
df = pd.read_excel('/content/GrabReviews_Clean.xlsx')

In [5]:
df.head()

,content,score,sentimen,clean_text,lower_text,normalized_text,tokenized_text,filtered_text,final_text,sentiment_score,sentiment_label
0,"gak jelassss,, cari lokasi jemput harus susah ...",1,negative,gak jelassss cari lokasi jemput harus susah ny...,gak jelassss cari lokasi jemput harus susah ny...,tidak jelassss cari lokasi jemput harus susah ...,"['tidak', 'jelassss', 'cari', 'lokasi', 'jempu...","['jelassss', 'cari', 'lokasi', 'jemput', 'susa...",jelassss cari lokasi jemput susah ampun,-5,negative
1,enak sangat memuaskan kan reccoment banget,5,positive,enak sangat memuaskan kan reccoment banget,enak sangat memuaskan kan reccoment banget,enak sangat memuaskan kan reccoment banget,"['enak', 'sangat', 'memuaskan', 'kan', 'reccom...","['enak', 'memuaskan', 'reccoment']",enak memuaskan reccoment,10,positive
2,sangat membantu,5,positive,sangat membantu,sangat membantu,sangat membantu,"['sangat', 'membantu']",['membantu'],membantu,4,positive
3,game bus telolet,3,neutral,game bus telolet,game bus telolet,game bus telolet,"['game', 'bus', 'telolet']","['game', 'bus', 'telolet']",game bus telolet,2,neutral
4,aku sih ok banget pelayanan nya,5,positive,aku sih ok banget pelayanan nya,aku sih ok banget pelayanan nya,saya sih baik banget pelayanan nya,"['saya', 'sih', 'baik', 'banget', 'pelayanan',...",['pelayanan'],pelayanan,3,positive


Split Data

In [6]:
texts = df['final_text'].astype(str).fillna('')
targets = df['sentiment_label']

from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

labeler = LabelEncoder()
targets_num = labeler.fit_transform(targets)

targets_onehot = to_categorical(targets_num)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train_num, y_test_num = train_test_split(
    texts, targets_num,
    test_size=0.20,
    random_state=123,
    stratify=targets_num
)

y_train = to_categorical(y_train_num)
y_test  = to_categorical(y_test_num)


In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [8]:
tok = Tokenizer(num_words=10000, oov_token="<UNK>")
tok.fit_on_texts(X_train)

train_seq = tok.texts_to_sequences(X_train)
test_seq = tok.texts_to_sequences(X_test)

max_length = 40

train_pad = pad_sequences(train_seq, maxlen=max_length, padding='post')
test_pad = pad_sequences(test_seq, maxlen=max_length, padding='post')

print("=== Contoh Hasil Tokenisasi & Padding ===")
print("Kalimat asli:\n", X_train[0])
print("\nHasil tokenisasi:\n", train_seq[0])
print("\nSetelah padding:\n", train_pad[0])

=== Contoh Hasil Tokenisasi & Padding ===
Kalimat asli:
 jelassss cari lokasi jemput susah ampun

Hasil tokenisasi:
 [41, 47]

Setelah padding:
 [41 47  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


In [9]:
print(f"Ukuran train_pad : {train_pad.shape}")
print(f"Ukuran y_train   : {y_train.shape}")
print(f"Ukuran test_pad  : {test_pad.shape}")
print(f"Ukuran y_test    : {y_test.shape}")

Ukuran train_pad : (12000, 40)
Ukuran y_train   : (12000, 3)
Ukuran test_pad  : (3000, 40)
Ukuran y_test    : (3000, 3)


In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, GlobalMaxPooling1D
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report
import numpy as np

#MODELLING MODEL 1: CNN–LSTM

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Conv1D, MaxPooling1D, Bidirectional, GlobalMaxPooling1D
from tensorflow.keras import regularizers

def create_cnn_lstm_model(vocab_size, embedding_dim, max_length):
    model = Sequential([
        Embedding(vocab_size, embedding_dim, input_length=max_length),
        Conv1D(filters=128, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Bidirectional(LSTM(64, dropout=0.3, return_sequences=True)),
        GlobalMaxPooling1D(),
        Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
        Dropout(0.5),
        Dense(3, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [15]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(texts)

vocab_size = len(tokenizer.word_index) + 1

max_length = max([len(x.split()) for x in texts])

embedding_dim = 100


In [19]:
vocab_size = len(tok.word_index) + 1
embedding_dim = 100

model = create_cnn_lstm_model(vocab_size, embedding_dim, max_length)

from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    train_pad, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(test_pad, y_test),
    callbacks=[early_stop]
)


Epoch 1/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 21s 42ms/step - accuracy: 0.6998 - loss: 1.0501 - val_accuracy: 0.9490 - val_loss: 0.2618
Epoch 2/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 15s 39ms/step - accuracy: 0.9554 - loss: 0.2323 - val_accuracy: 0.9593 - val_loss: 0.2016
Epoch 3/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 14s 38ms/step - accuracy: 0.9676 - loss: 0.1678 - val_accuracy: 0.9627 - val_loss: 0.1880
Epoch 4/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 21s 39ms/step - accuracy: 0.9679 - loss: 0.1575 - val_accuracy: 0.9653 - val_loss: 0.1819
Epoch 5/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 15s 40ms/step - accuracy: 0.9681 - loss: 0.1572 - val_accuracy: 0.9627 - val_loss: 0.1988
Epoch 6/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 20s 40ms/step - accuracy: 0.9690 - loss: 0.1448 - val_accuracy: 0.9657 - val_loss: 0.1811
Epoch 7/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 14s 38ms/step - accuracy: 0.9691 - loss: 0.1434 - val_accuracy: 0.9670 - val_loss: 0.1854
Epoch 8/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 15s 39ms/step - accuracy: 0.9667 - loss: 0.1418 - 

In [21]:
loss, acc = model.evaluate(test_pad, y_test)
print(f"Akurasi Data Testing: {acc*100:.2f}%")


94/94 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.9650 - loss: 0.1716
Akurasi Data Testing: 96.57%


#MODELLING MODEL 2 - GRU

In [28]:
from tensorflow.keras.layers import GRU

def create_gru_model(vocab_size, embedding_dim, max_length):
    model = Sequential([
        Embedding(vocab_size, embedding_dim, input_length=max_length),
        Bidirectional(GRU(64, return_sequences=True, dropout=0.3)),
        GlobalMaxPooling1D(),
        Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
        Dropout(0.5),
        Dense(3, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [29]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

class_weights_array = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train_num),
    y=y_train_num
)

class_weights = dict(enumerate(class_weights_array))

print("Class Weights:", class_weights)


Class Weights: {0: np.float64(2.867383512544803), 1: np.float64(0.7676069852235655), 2: np.float64(0.7415647015202076)}


In [31]:
model = create_gru_model(vocab_size, embedding_dim, max_length)
history = model.fit(
    train_pad, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(test_pad, y_test),
    callbacks=[early_stop],
    class_weight=class_weights
)

Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


375/375 ━━━━━━━━━━━━━━━━━━━━ 30s 68ms/step - accuracy: 0.6929 - loss: 1.0817 - val_accuracy: 0.9383 - val_loss: 0.2883
Epoch 2/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 21s 56ms/step - accuracy: 0.9454 - loss: 0.2434 - val_accuracy: 0.9457 - val_loss: 0.2252
Epoch 3/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 20s 53ms/step - accuracy: 0.9592 - loss: 0.1769 - val_accuracy: 0.9463 - val_loss: 0.2113
Epoch 4/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 20s 52ms/step - accuracy: 0.9603 - loss: 0.1680 - val_accuracy: 0.9497 - val_loss: 0.2057
Epoch 5/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 21s 54ms/step - accuracy: 0.9632 - loss: 0.1393 - val_accuracy: 0.9557 - val_loss: 0.2033
Epoch 6/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 19s 51ms/step - accuracy: 0.9614 - loss: 0.1439 - val_accuracy: 0.9597 - val_loss: 0.1964
Epoch 7/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 20s 53ms/step - accuracy: 0.9632 - loss: 0.1380 - val_accuracy: 0.9460 - val_loss: 0.2078
Epoch 8/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 20s 54ms/step - accuracy: 0.9622 - loss: 0.1280 - val_accurac

In [27]:
loss, acc = model.evaluate(test_pad, y_test)
print(f"Akurasi Data Testing: {acc*100:.2f}%")

94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9614 - loss: 0.1823
Akurasi Data Testing: 96.10%


#MODELLING MODEL 3 - LSTM

In [32]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, GlobalMaxPooling1D
from tensorflow.keras import regularizers

def create_lstm_model(vocab_size, embedding_dim, max_length):
    model = Sequential([
        Embedding(vocab_size, embedding_dim, input_length=max_length),
        LSTM(128, dropout=0.3, return_sequences=True),
        GlobalMaxPooling1D(),
        Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
        Dropout(0.5),
        Dense(3, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [33]:
model3 = create_lstm_model(vocab_size, embedding_dim, max_length)

history3 = model3.fit(
    train_pad, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(test_pad, y_test),
    callbacks=[early_stop],
    class_weight=class_weights
)

Epoch 1/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 31s 72ms/step - accuracy: 0.6043 - loss: 1.1698 - val_accuracy: 0.9240 - val_loss: 0.3268
Epoch 2/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 42s 74ms/step - accuracy: 0.9387 - loss: 0.2939 - val_accuracy: 0.9540 - val_loss: 0.2238
Epoch 3/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 27s 71ms/step - accuracy: 0.9620 - loss: 0.2199 - val_accuracy: 0.9547 - val_loss: 0.2129
Epoch 4/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 26s 70ms/step - accuracy: 0.9547 - loss: 0.2029 - val_accuracy: 0.9577 - val_loss: 0.2028
Epoch 5/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 27s 71ms/step - accuracy: 0.9574 - loss: 0.1822 - val_accuracy: 0.9427 - val_loss: 0.2319
Epoch 6/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 27s 72ms/step - accuracy: 0.9547 - loss: 0.1739 - val_accuracy: 0.9503 - val_loss: 0.2139
Epoch 7/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 29s 77ms/step - accuracy: 0.9605 - loss: 0.1522 - val_accuracy: 0.9420 - val_loss: 0.2113


In [34]:
loss, acc = model.evaluate(test_pad, y_test)
print(f"Akurasi Data Testing: {acc*100:.2f}%")

94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9604 - loss: 0.1836
Akurasi Data Testing: 95.97%


Testing

In [57]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

kalimat_baru = input("Masukkan kalimat baru: ")

kalimat_baru_cleaned = clean_text(kalimat_baru)
kalimat_baru_casefolded = lowercase_text(kalimat_baru_cleaned)
kalimat_baru_slangfixed = convert_slang(kalimat_baru_casefolded)
kalimat_baru_tokenized = tokenize_text(kalimat_baru_slangfixed)
kalimat_baru_filtered = remove_stopwords(kalimat_baru_tokenized)
kalimat_baru_final = list_to_sentence(kalimat_baru_filtered)

X_kalimat_baru = tokenizer.texts_to_sequences([kalimat_baru_final])
X_kalimat_baru = pad_sequences(X_kalimat_baru, maxlen=40, padding='post', truncating='post')

prediksi_sentimen = model.predict(X_kalimat_baru)

predicted_class_index = np.argmax(prediksi_sentimen, axis=1)[0]

label_kategori = ["negatif", "netral", "positif"]
predicted_class_label = label_kategori[predicted_class_index]

# Output hasil
print(f"\nKalimat baru: \"{kalimat_baru}\"")
print(f"Prediksi Sentimen: {predicted_class_label.capitalize()} ({prediksi_sentimen[0][predicted_class_index]*100:.2f}%)")


Masukkan kalimat baru: Sangat membantu dalam bepergian dan harga murah
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step

Kalimat baru: "Sangat membantu dalam bepergian dan harga murah"
Prediksi Sentimen: Positif (99.68%)


In [59]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

kalimat_baru = input("Masukkan kalimat baru: ")

kalimat_baru_cleaned = clean_text(kalimat_baru)
kalimat_baru_casefolded = lowercase_text(kalimat_baru_cleaned)
kalimat_baru_slangfixed = convert_slang(kalimat_baru_casefolded)
kalimat_baru_tokenized = tokenize_text(kalimat_baru_slangfixed)
kalimat_baru_filtered = remove_stopwords(kalimat_baru_tokenized)
kalimat_baru_final = list_to_sentence(kalimat_baru_filtered)

X_kalimat_baru = tokenizer.texts_to_sequences([kalimat_baru_final])
X_kalimat_baru = pad_sequences(X_kalimat_baru, maxlen=40, padding='post', truncating='post')

prediksi_sentimen = model.predict(X_kalimat_baru)

predicted_class_index = np.argmax(prediksi_sentimen, axis=1)[0]

label_kategori = ["negatif", "netral", "positif"]
predicted_class_label = label_kategori[predicted_class_index]

# Output hasil
print(f"\nKalimat baru: \"{kalimat_baru}\"")
print(f"Prediksi Sentimen: {predicted_class_label.capitalize()} ({prediksi_sentimen[0][predicted_class_index]*100:.2f}%)")


Masukkan kalimat baru: Aplikasi susah digunakan
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step

Kalimat baru: "Aplikasi susah digunakan"
Prediksi Sentimen: Negatif (89.86%)


In [60]:
!pip freeze > requirements.txt

In [61]:
from google.colab import files
files.download('requirements.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>